In [46]:
import pandas as pd
import numpy as np

In [47]:
weights = {
    'AMXB': 0.08,
    'CEMEX': 0.11,
    'GAPB': 0.12,
    'LACOMER': 0.09,
    'PENOLES': 0.21,
    'BANREGIO': 0.14,
    'VOLARIS': 0.17,
    'FIBRAPL': 0.08
}

In [48]:
portfolio_value = 350_000_000  # 350 millones de pesos

In [49]:
file_path = "Data_C2.xlsx"

returns = []

for asset, w in weights.items():
    
    df = pd.read_excel(
        file_path,
        sheet_name=asset,
        header=2
    )
    
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date')
    
    df['Return'] = df['Price'].pct_change().dropna()
    
    df = df[['Date', 'Return']].dropna()
    df = df.rename(columns={'Return': asset})
    
    returns.append(df.set_index('Date'))

In [50]:
returns_df = pd.concat(returns, axis=1).dropna()
returns_df.head()

,AMXB,CEMEX,GAPB,LACOMER,PENOLES,BANREGIO,VOLARIS,FIBRAPL
Date,,,,,,,,
2024-01-29,0.001293,0.009894,0.012080,0.001970,0.072147,-0.005445,0.025018,-0.019874
2024-01-30,0.005165,0.006298,-0.014657,-0.001474,-0.026984,0.016298,-0.032078,-0.013410
2024-01-31,-0.001285,-0.005563,-0.013439,-0.005415,-0.025321,-0.010897,-0.008646,0.025000
2024-02-01,-0.003859,0.008392,0.005673,0.004454,-0.006406,0.001315,0.000727,-0.011138
2024-02-02,0.001291,0.020111,0.036627,-0.001232,-0.035573,0.012253,0.006536,0.012546


In [51]:
weights_array = np.array(list(weights.values()))
returns_df['Portfolio_Return'] = np.dot(returns_df, weights_array)

#### **1) Calcular el VaR utilizando el método de simulación histórica con 99% de confianza**

In [52]:
VaR_99 = -np.percentile(returns_df['Portfolio_Return'], 1)
VaR_99 * 100, VaR_99 * portfolio_value

(np.float64(2.8611433660175476), np.float64(10014001.781061416))

#### **2) Calcular el VaR utilizando a 95% de confianza**

In [53]:
VaR_95 = -np.percentile(returns_df['Portfolio_Return'], 5)
VaR_95 * 100, VaR_95 * portfolio_value

(np.float64(1.9799543673876623), np.float64(6929840.285856819))

#### **3) Calcular ndays para cada posición de la inversión**

In [54]:
vol = []

for asset, w in weights.items():
    
    df = pd.read_excel(
        file_path,
        sheet_name=asset,
        header=2
    )
    
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date')
    
    last_price = df['Price'].iloc[-1]
    mean_volume = np.floor((df['CVol'].tail(90).mean())*0.1)
    titles_to_trade = np.floor((w * portfolio_value) / last_price)
    ndays = np.round(titles_to_trade / mean_volume,2)
    
    vol.append({
        "last_price": last_price,
        'position_value': w * portfolio_value,
        "mean_volume": mean_volume,
        "titles_to_trade": titles_to_trade,
        "ndays": ndays,
        'líquido?': 'Sí' if ndays < 5 else 'No'
    })

vol_df = pd.DataFrame(vol, index=weights.keys())
vol_df

,last_price,position_value,mean_volume,titles_to_trade,ndays,líquido?
AMXB,17.89,28000000.0,5118143.0,1565120.0,0.31,Sí
CEMEX,22.10,38500000.0,2837725.0,1742081.0,0.61,Sí
GAPB,486.29,42000000.0,86424.0,86368.0,1.00,Sí
LACOMER,38.18,31500000.0,93662.0,825039.0,8.81,No
PENOLES,1109.20,73500000.0,66243.0,66263.0,1.00,Sí
BANREGIO,155.19,49000000.0,85821.0,315741.0,3.68,Sí
VOLARIS,16.37,59500000.0,240605.0,3634697.0,15.11,No
FIBRAPL,80.35,28000000.0,133918.0,348475.0,2.60,Sí
